In [1]:
import firefin
import pandas as pd
import numpy as np

In [2]:
# load data
data = firefin.fetch_data(['market_cap', 'return_adj', 'pb_ratio'])
mkt_cap = data['market_cap'].iloc[:60, :100].dropna(axis = 1)
ret_adj = data['return_adj'].iloc[:60, :100].dropna(axis = 1)
pb = data['pb_ratio'].iloc[:60, :100].dropna(axis = 1)
bm = 1 / pb

/Users/nicho/Github/fire/firefin/data/gateway.py:85: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(obj)
/Users/nicho/Github/fire/firefin/data/gateway.py:85: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(obj)


In [30]:
risk_free_rate = pd.read_feather('/Users/nicho/Github/bond_data/us_bond_2y.feather')

In [4]:
dates = ret_adj.index.normalize()
risk_free_rate = risk_free_rate.loc[dates].ffill() / 100

mkt_cap.index = dates
ret_adj.index = dates
bm.index = dates
risk_free_rate.index = dates

In [ ]:
from calculate_ff3_factors import cal_ff3

In [6]:
smb =[]
hml = []
mkt_ret = []
for date in dates:
    stock_return_date = ret_adj.loc[date]
    stock_size_date = mkt_cap.loc[date]
    stock_value_date = bm.loc[date]
    risk_free_rate_date = risk_free_rate.loc[date, 'us_bond_2y']

    smb_date, hml_date, mkt_ret_date = cal_ff3(
        stock_return_date, 
        stock_size_date,
        stock_value_date,
        risk_free_rate_date
    )

    smb.append(smb_date)
    hml.append(hml_date)
    mkt_ret.append(risk_free_rate_date)


In [ ]:
smb_df = pd.DataFrame(index = dates, columns = ret_adj.columns)
smb_df[:] = np.array(smb).reshape(-1, 1)
smb_df = smb_df.astype(float)

hml_df = pd.DataFrame(index = dates, columns = ret_adj.columns)
hml_df[:] = np.array(hml).reshape(-1, 1)
hml_df = hml_df.astype(float)

mkt_ret_df = pd.DataFrame(index = dates, columns = ret_adj.columns)
mkt_ret_df[:] = np.array(mkt_ret).reshape(-1, 1)
mkt_ret_df = mkt_ret_df.astype(float)

In [8]:
ff3_factors = np.stack([
    smb_df.values,
    hml_df.values,
    mkt_ret_df.values
], axis = 0)

In [9]:
excess_ret = ret_adj.sub(risk_free_rate['us_bond_2y'], axis = 0)

In [10]:
from firefin.core.algorithm.regression import rolling_regression, table_regression

In [11]:
window = 50
result = rolling_regression(
    x = ff3_factors, 
    y = excess_ret,
    window = window
)

In [33]:
result.alpha.tail()

stock_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,000418.SZ,000419.SZ,000420.SZ,000421.SZ,000422.SZ,000423.SZ,000425.SZ,000426.SZ,000428.SZ,000429.SZ
datetime,,,,,,,,,,,,,,,,,,,,,
2009-03-30,0.031051,0.006178,-0.010422,0.075783,-0.032807,0.012822,0.001591,0.117516,0.025407,0.013076,...,0.020493,-0.031444,0.036445,-0.004768,0.022996,0.019262,-0.015598,-0.035386,0.009523,0.017130
2009-03-31,0.046227,0.021477,0.005374,0.097443,-0.011408,0.031858,0.025740,0.139097,0.036308,0.033344,...,0.028564,-0.001700,0.064143,0.020704,0.044761,0.039839,-0.003255,-0.014261,0.032735,0.022587
2009-04-01,0.023536,0.010278,0.012644,0.090597,-0.020219,0.027979,0.012867,0.131473,0.032273,0.025330,...,0.025735,-0.014849,0.056705,0.007095,0.032206,0.030644,-0.020103,-0.025721,0.023097,0.020089
2009-04-02,0.028247,0.018064,0.016281,0.094499,-0.019613,0.027329,0.024109,0.120383,0.033557,0.028743,...,0.040946,-0.015179,0.062338,0.007709,0.023266,0.027965,-0.025213,-0.029453,0.019418,0.022742
2009-04-03,0.023403,0.020929,0.020327,0.102587,-0.023275,0.027140,0.031788,0.114440,0.023853,0.032387,...,0.041372,-0.030529,0.077432,0.007685,0.045947,0.047830,-0.026913,-0.031762,0.014615,0.023287


In [34]:
result.beta[0].tail()

stock_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,000418.SZ,000419.SZ,000420.SZ,000421.SZ,000422.SZ,000423.SZ,000425.SZ,000426.SZ,000428.SZ,000429.SZ
datetime,,,,,,,,,,,,,,,,,,,,,
2009-03-30,-1.120872,-1.116252,0.053869,-0.202427,-1.204555,0.728096,0.570517,-1.759960,0.329999,0.379739,...,-0.900397,1.234492,0.553316,0.512136,-0.039121,-0.186154,-0.931722,1.135240,-0.291596,-0.356717
2009-03-31,-1.013166,-1.088883,0.158422,-0.226277,-1.270790,0.796385,0.595272,-1.935596,0.415546,0.472173,...,-1.007811,1.220376,0.472508,0.428027,-0.078409,-0.294116,-0.891259,1.020645,-0.272760,-0.378183
2009-04-01,-0.917088,-1.043326,0.114172,-0.178476,-1.219642,0.812831,0.661600,-1.873430,0.442405,0.508866,...,-1.000021,1.285694,0.524154,0.504930,-0.019023,-0.249743,-0.810902,1.080859,-0.215994,-0.362426
2009-04-02,-0.927655,-1.022180,0.235236,-0.078086,-1.121638,0.839952,0.614133,-1.762246,0.505183,0.567494,...,-0.967084,1.340554,0.489475,0.549463,0.094976,-0.321215,-0.764553,1.139583,-0.148081,-0.308219
2009-04-03,-0.929918,-1.020765,0.210700,-0.128094,-1.137708,0.848748,0.545427,-1.829265,0.458466,0.576941,...,-0.816725,1.301346,0.379677,0.481606,0.006366,-0.393227,-0.737412,1.047375,-0.230931,-0.324353


In [35]:
result.beta[1].tail()

stock_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,000418.SZ,000419.SZ,000420.SZ,000421.SZ,000422.SZ,000423.SZ,000425.SZ,000426.SZ,000428.SZ,000429.SZ
datetime,,,,,,,,,,,,,,,,,,,,,
2009-03-30,1.510347,1.562336,0.966974,0.814770,2.524961,1.021402,1.221902,1.682733,0.708520,0.833634,...,1.554249,1.933119,1.883062,1.721345,2.310736,0.736669,0.280139,2.096553,1.673936,1.268529
2009-03-31,1.538624,1.572651,0.994715,0.815353,2.515978,1.041906,1.234053,1.649294,0.730665,0.859883,...,1.532396,1.938093,1.872538,1.709472,2.307890,0.718114,0.292579,2.076663,1.684505,1.265214
2009-04-01,1.421811,1.519374,1.063802,0.735887,2.438087,1.021886,1.139991,1.539793,0.686918,0.812139,...,1.527681,1.847736,1.786878,1.594089,2.228621,0.657991,0.184644,1.990187,1.597379,1.240181
2009-04-02,1.433733,1.522757,1.018392,0.699634,2.397626,1.009395,1.177842,1.475252,0.662417,0.792762,...,1.537832,1.824029,1.810450,1.576239,2.166291,0.683959,0.156968,1.959463,1.562855,1.221469
2009-04-03,1.425382,1.527702,1.023754,0.710295,2.390436,1.009610,1.186671,1.461088,0.643114,0.799517,...,1.547726,1.795611,1.829342,1.572060,2.199342,0.713246,0.155741,1.949923,1.549658,1.221410


In [36]:
result.beta[2].tail()

stock_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,000418.SZ,000419.SZ,000420.SZ,000421.SZ,000422.SZ,000423.SZ,000425.SZ,000426.SZ,000428.SZ,000429.SZ
datetime,,,,,,,,,,,,,,,,,,,,,
2009-03-30,-3.232633,-1.199728,0.850752,-8.431459,3.820845,-1.556923,-0.311481,-12.174225,-2.670137,-1.268631,...,-2.539634,3.460976,-4.111060,0.297233,-2.541337,-2.845239,1.600024,4.206138,-0.495865,-2.370411
2009-03-31,-4.787840,-2.758985,-0.767283,-10.632567,1.650719,-3.500730,-2.770819,-14.351247,-3.788221,-3.340232,...,-3.349475,0.436524,-6.920376,-2.285279,-4.751387,-4.927258,0.340041,2.069117,-2.859255,-2.923295
2009-04-01,-2.466003,-1.609781,-1.486975,-9.965703,2.527548,-3.103874,-1.474833,-13.624743,-3.392843,-2.525166,...,-3.052467,1.764737,-6.195320,-0.927325,-3.477905,-3.996119,2.047833,3.220793,-1.901644,-2.676920
2009-04-02,-2.947193,-2.424390,-1.923455,-10.419170,2.415668,-3.050374,-2.611870,-12.535407,-3.556981,-2.907105,...,-4.639856,1.771183,-6.759673,-1.013071,-2.611903,-3.683544,2.552228,3.576613,-1.555905,-2.978133
2009-04-03,-2.459339,-2.712870,-2.372697,-11.318756,2.761807,-3.017256,-3.499636,-12.038484,-2.646347,-3.261906,...,-4.445205,3.266515,-8.464597,-1.117996,-5.053169,-5.812773,2.767660,3.665056,-1.199669,-3.059000


In [12]:
smb_beta = result.beta[0].iloc[(window - 1):]
hml_beta = result.beta[1].iloc[(window - 1):]
mkt_ret_beta = result.beta[2].iloc[(window - 1):]

In [13]:
beta = np.stack([
    smb_beta.values,
    hml_beta.values,
    mkt_ret_beta.values
], axis = 0)

In [14]:
result_2 = table_regression(x = beta, y = excess_ret.iloc[(window - 1):])

In [15]:
result_2.alpha

datetime
2009-03-20   -0.019942
2009-03-23   -0.006392
2009-03-24   -0.005620
2009-03-25   -0.008146
2009-03-26   -0.005355
2009-03-27   -0.004864
2009-03-30    0.007397
2009-03-31   -0.010675
2009-04-01   -0.012938
2009-04-02   -0.012264
2009-04-03   -0.007563
Name: alpha, dtype: float64

In [22]:
result_2.beta

,0,1,2
datetime,,,
2009-03-20,0.010607,0.004425,-0.000256
2009-03-23,-0.005544,0.010589,-0.000592
2009-03-24,-0.005059,0.001975,0.000106
2009-03-25,0.006957,-0.007458,0.002341
2009-03-26,0.001894,0.010336,-0.000936
2009-03-27,-0.000402,-0.001485,-0.001640
2009-03-30,0.010615,-0.005975,-0.001289
2009-03-31,-0.011883,-0.002119,-0.002994
2009-04-01,0.000174,0.012924,-0.000322


In [16]:
result_2.beta[0]

datetime
2009-03-20    0.010607
2009-03-23   -0.005544
2009-03-24   -0.005059
2009-03-25    0.006957
2009-03-26    0.001894
2009-03-27   -0.000402
2009-03-30    0.010615
2009-03-31   -0.011883
2009-04-01    0.000174
2009-04-02   -0.011307
2009-04-03    0.002319
Name: 0, dtype: float64

In [17]:
result_2.beta[1]

datetime
2009-03-20    0.004425
2009-03-23    0.010589
2009-03-24    0.001975
2009-03-25   -0.007458
2009-03-26    0.010336
2009-03-27   -0.001485
2009-03-30   -0.005975
2009-03-31   -0.002119
2009-04-01    0.012924
2009-04-02    0.003643
2009-04-03   -0.001600
Name: 1, dtype: float64

In [18]:
result_2.beta[2]

datetime
2009-03-20   -0.000256
2009-03-23   -0.000592
2009-03-24    0.000106
2009-03-25    0.002341
2009-03-26   -0.000936
2009-03-27   -0.001640
2009-03-30   -0.001289
2009-03-31   -0.002994
2009-04-01   -0.000322
2009-04-02    0.000283
2009-04-03    0.000778
Name: 2, dtype: float64